<a href="https://colab.research.google.com/github/garrettsomers/ManningLLMBook/blob/chapter8/Chapter_8_Retreival_Augmented_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install llama-index-finetuning==0.1.0 -q -U
%pip install llama-index-embeddings-openai==0.1.1 -q -U
%pip install llama-index-embeddings-huggingface==0.1.1 -q -U
%pip install llama-index-llms-openai==0.1.0 -q -U
%pip install openai==1.12.0 -q -U
%pip install sentence_transformers -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.9/622.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
import nest_asyncio
import getpass
from google.colab import drive
import locale
locale.getpreferredencoding = lambda: "UTF-8"

from tqdm import tqdm
import pandas as pd

# Turn on async
nest_asyncio.apply()

# Enter OpenAI API key
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

Mounted at /content/drive
Archive:  /content/drive/MyDrive/llm-book-data/openai_tos.zip
   creating: ./openai_tos/
  inflating: ./openai_tos/09_sharing-and-publication-policy.txt  
  inflating: ./openai_tos/02_privacy-policy.txt  
  inflating: ./openai_tos/07_usage-policies.txt  
  inflating: ./openai_tos/05_plugins-and-actions-terms.txt  
  inflating: ./openai_tos/08_enterprise-privacy-at-openai.txt  
  inflating: ./openai_tos/06_business-terms.txt  
  inflating: ./openai_tos/01_terms-of-use.txt  
  inflating: ./openai_tos/03_service-terms.txt  
  inflating: ./openai_tos/04_data-processing-addendum.txt  
  inflating: ./openai_tos/10_coordinated-vulnerability-disclosure-policy.txt  


In [ ]:
from tqdm import tqdm
import pandas as pd
import torch

from llama_index.core import (
    Document,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Basic RAG system

In [ ]:
## Collect the OpenAI TOS docs
!unzip ../data/openai_tos.zip ../data/openai_tos
directory = '../data/openai_tos/'
doc_names = sorted(os.listdir(directory))

## Collect each TOS doc into a llama_index Document object
documents = []
for i, doc_name in enumerate(doc_names):
  document = open(directory+doc_name).read()
  d = Document(
      text=document,
      metadata = {"file": doc_name, "name": doc_name.split('_')[1].split('.')[0].replace('-',' ')}
  )
  documents.append(d)

In [ ]:
# Define a text chunking procedure
text_chunker = SentenceSplitter(chunk_size=128, chunk_overlap=8)
# Split the documnets into nodes
nodes = text_chunker.get_nodes_from_documents(documents)
# Load a model for embedding the text
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

In [ ]:
output_dir = '../data/bge-small-en-v1.5_openai-tos_vectors/'

In [ ]:
## Generate indexing vectors

if True:
    index = VectorStoreIndex(
        nodes,
        embed_model=embed_model,
        show_progress=True
    )

    ## Save embeddings with a storage context.
    index.storage_context.persist(persist_dir = output_dir)

## Load embeds from storage context. Requires setting the same storage_context as when generated.
else:
    storage_context = StorageContext.from_defaults(persist_dir = output_dir)
    index = load_index_from_storage(
        storage_context=storage_context,
        embed_model=embed_model
    )

In [ ]:
## Create a simple query engine and compare to GPT responses

def gpt_and_rag_answers(query,query_engine,llm_engine):
    llm_response = llm_engine.complete(query)
    rag_response = query_engine.query(query)

    print(f'############\nLLM response:\n{llm_response}\n\n############\nRAG response:\n{rag_response}')
    return llm_response, rag_response

## Create a GPT-4 object for API calls
llm_gpt = OpenAI(model='gpt-4',temperature=0.0)
query_engine = index.as_query_engine(llm=llm_gpt)

In [ ]:
#query = 'Who owns the content created on OpenAI resources?'
query = 'Who owns the content created by OpenAI programs?'
gpt, rag = gpt_and_rag_answers(query, query_engine, llm_gpt)

############
LLM response:
OpenAI retains the ownership of the content created by its programs. However, the user who uses the program to generate content has the rights to that specific output. OpenAI does not claim any ownership over the content generated by users through its programs.

############
RAG response:
The content created by OpenAI programs is owned by the user. OpenAI assigns all its right, title, and interest, if any, in and to the output to the user.


# Improved RAG Querying

In [ ]:
from llama_index.core import get_response_synthesizer, PromptTemplate
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import LLMRerank
from llama_index.core.response_synthesizers import ResponseMode

In [ ]:
## Create improved querying engine, and querying engine with re-ranking

TopK = 5

Rerank_TopK = 20
Rerank_TopRRK = 5

qa_prompt_tmpl = PromptTemplate(
    "You are an expert Q&A system that is trusted around the world."
    "Always answer the query using the provided context information, and not prior knowledge.\n"
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, answer the query.\n"
    "Query: {query_str}\n"
    "Answer: "
)

###################

retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=TopK,
)

response_synthesizer = get_response_synthesizer(response_mode=ResponseMode.SIMPLE_SUMMARIZE)

custom_query_engine = RetrieverQueryEngine.from_args(
    retriever,
    response_synthesizer=response_synthesizer,
)
custom_query_engine.update_prompts(
     {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

###################

retriever_rr = VectorIndexRetriever(
    index=index,
    similarity_top_k=Rerank_TopK,
)

response_synthesizer_rr = get_response_synthesizer(response_mode=ResponseMode.SIMPLE_SUMMARIZE)

custom_query_engine_rerank = RetrieverQueryEngine.from_args(
    retriever_rr,
    response_synthesizer=response_synthesizer_rr,
    node_postprocessors=[
        LLMRerank(
            choice_batch_size=5,
            top_n=Rerank_TopRRK,
        )],
)
custom_query_engine_rerank.update_prompts(
     {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [ ]:
query = 'Will OpenAI give me any sort of reward if I find an issue with their software?'

gpt, rag = gpt_and_rag_answers(query, custom_query_engine, llm_gpt)

############
LLM response:
OpenAI has not publicly stated that they offer rewards for finding issues with their software. However, they would likely appreciate the feedback to help improve their systems. It's always a good idea to reach out to them directly for the most accurate information.

############
RAG response:
No, OpenAI does not mention providing rewards for finding issues with their software in the provided context information.


In [ ]:
query = 'Will OpenAI give me any sort of reward if I find an issue with their software?'

gpt, rag_rr = gpt_and_rag_answers(query, custom_query_engine_rerank, llm_gpt)

############
LLM response:
OpenAI has not publicly announced any bug bounty programs or rewards for finding issues in their software. However, they would likely appreciate being informed about any potential issues or bugs. You can contact them through their official website or other official communication channels. Always remember to follow responsible disclosure guidelines when reporting potential issues.

############
RAG response:
Yes, OpenAI offers a Bug Bounty Program as a way to recognize and reward security researchers who contribute to keeping their technology and company secure.


# Fine-tuning

In [ ]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset
from sklearn.model_selection import train_test_split

In [ ]:
train_nodes, val_nodes = train_test_split(pd.Series(nodes), test_size=0.20, random_state=7)
print(len(train_nodes), len(val_nodes))

260 66


In [ ]:
json_locs = '../data/bge-small-en-v1.5_openai-tos_qa-embedding-pairs/'
!mkdir $json_locs

if True:
    train_dataset = generate_qa_embedding_pairs(
        train_nodes,
        llm=OpenAI(model='gpt-3.5-turbo',temperature=0.0),
    )
    val_dataset = generate_qa_embedding_pairs(
        val_nodes,
        llm=OpenAI(model='gpt-3.5-turbo',temperature=0.0),
    )

    train_dataset.save_json(json_locs+"train_dataset.json")
    val_dataset.save_json(json_locs+"val_dataset.json")

else:
    train_dataset = EmbeddingQAFinetuneDataset.from_json(json_locs+"train_dataset.json")
    val_dataset = EmbeddingQAFinetuneDataset.from_json(json_locs+"val_dataset.json")

In [ ]:
## Look at validation set queries and find an interesting example
val_dataset.__dict__['queries']

In [ ]:
query_id = '0222418f-4e63-4577-84ec-94597552bff4'
corpus_id = val_dataset.__dict__['relevant_docs'][query_id][0]

sample_chunk = val_dataset.__dict__['corpus'][corpus_id]
sample_query = val_dataset.__dict__['queries'][query_id]

print(f'Sample text chunk:\n------------------\n{sample_chunk}')
print(f'\n\nQuery based on chunk:\n---------------------\n{sample_query}')

Sample text chunk:
------------------
If you notice that ChatGPT output contains factually inaccurate information about you and you would like us to correct the inaccuracy, you may submit a correction request through privacy.openai.com or to dsar@openai.com. Given the technical complexity of how our models work, we may not be able to correct the inaccuracy in every instance. In that case, you may request that we remove your Personal Information from ChatGPT’s output by filling out this form.


Query based on chunk:
---------------------
How can individuals request corrections for factually inaccurate information about themselves in ChatGPT output?


In [ ]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine

model_ft_path = '../data/bge-small-en-v1.5_openai-tos_finetuned-model'

if False:
    finetune_engine = SentenceTransformersFinetuneEngine(
        model_id = "BAAI/bge-small-en-v1.5",
        dataset = train_dataset,
        val_dataset = val_dataset,
        epochs = 4,
        model_output_path = model_ft_path
    )
    finetune_engine.finetune()
    finetuned_embedding_model = finetune_engine.get_finetuned_model()

else:
    finetuned_embedding_model = HuggingFaceEmbedding(
        model_name = model_ft_path
    )

In [ ]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer
from pathlib import Path

def evaluate_st(dataset, model_id, name):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs, name=name)
    model = SentenceTransformer(model_id)
    output_path = "results/"
    Path(output_path).mkdir(exist_ok=True, parents=True)
    return evaluator(model, output_path=output_path)

In [ ]:
evaluate_st(val_dataset, "BAAI/bge-small-en-v1.5", name="bge")

0.9361111111111111

In [ ]:
evaluate_st(val_dataset, model_ft_path, name="finetuned")

0.9551767676767677

In [ ]:
output_dir = '../data/bge-small-en-v1.5_openai-tos_vectors'

if True:
    ## Generate new vector-index with the fine-tuned embedding model
    index_ft = VectorStoreIndex(
        nodes,
        embed_model = finetuned_embedding_model,
        show_progress=True
    )

    ## Save embeddings with a storage context.
    index_ft.storage_context.persist(persist_dir = output_dir)

else: ## Load vectors
    storage_context = StorageContext.from_defaults(persist_dir = output_dir)
    index_ft = load_index_from_storage(
        storage_context=storage_context,
        embed_model=finetuned_embedding_model
    )

Generating embeddings:   0%|          | 0/326 [00:00<?, ?it/s]

In [ ]:
ft_query_engine = index_ft.as_query_engine()

def ft_gpt_and_rag_answers(query,query_engine,llm_engine):
    #gpt_response = OpenAI(model=gpt_model).complete(query)
    llm_response = llm_engine.complete(query)
    rag_response = query_engine.query(query)

    print(f'############\nLLM response:\n{llm_response}\n\n############\nRAG response:\n{rag_response}')
    return llm_response, rag_response

In [ ]:
## Create

TopK = 5

Rerank_TopK = 20
Rerank_TopRRK = 5

qa_prompt_tmpl = PromptTemplate(
    "You are an expert Q&A system that is trusted around the world."
    "Always answer the query using the provided context information, and not prior knowledge.\n"
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information and not prior knowledge, answer the query.\n"
    "Query: {query_str}\n"
    "Answer: "
)

###################

retriever_ft = VectorIndexRetriever(
    index=index_ft,
    similarity_top_k=TopK,
)

response_synthesizer_ft = get_response_synthesizer(response_mode=ResponseMode.SIMPLE_SUMMARIZE)

custom_query_engine_ft = RetrieverQueryEngine.from_args(
    retriever_ft,
    response_synthesizer=response_synthesizer_ft,
)
custom_query_engine_ft.update_prompts(
     {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

###################

retriever_rr_ft = VectorIndexRetriever(
    index=index_ft,
    similarity_top_k=Rerank_TopK,
)

response_synthesizer_rr_ft = get_response_synthesizer(response_mode=ResponseMode.SIMPLE_SUMMARIZE)

custom_query_engine_rerank_ft = RetrieverQueryEngine.from_args(
    retriever_rr_ft,
    response_synthesizer=response_synthesizer_rr_ft,
    node_postprocessors=[
        LLMRerank(
            choice_batch_size=5,
            top_n=Rerank_TopRRK,
        )],
)
custom_query_engine_rerank_ft.update_prompts(
     {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

In [ ]:
query = 'How does ChatGPT deal with inappropriate questions?'

custom_query_engine.query(query).response

'ChatGPT ensures that automated systems, such as chatbots, disclose to people that they are interacting with AI unless it is obvious from the context. Additionally, ChatGPT does not build tools that may be inappropriate for minors, including sexually explicit or suggestive content, unless it is created for scientific or educational purposes.'

In [ ]:
custom_query_engine_ft.query(query).response

'ChatGPT deals with inappropriate questions by not allowing the creation of tools that may be inappropriate for minors, such as sexually explicit or suggestive content. Additionally, ChatGPT prohibits generating or promoting disinformation, misinformation, or false online engagement, impersonating another individual or organization without consent, engaging in or promoting academic dishonesty, using content from third parties without necessary permissions, and misrepresenting or misleading others about the purpose of the GPT.'

# Evaluation

## Context Relevance -- Is the retrieved context relevant to the query?

In [ ]:
from llama_index.core.evaluation import RetrieverEvaluator

def run_context_relevance_eval(index, queries, expected_ids, rerank=False):
    if rerank:
        retriever = VectorIndexRetriever(
            index=index,
            similarity_top_k=20,
            node_postprocessors=[
                LLMRerank(
                    choice_batch_size=5,
                    top_n=2,
                )],
        )
    else:
        retriever = index.as_retriever(similarity_top_k=2)

    retriever_evaluator = RetrieverEvaluator.from_metric_names(
        ["mrr", "hit_rate"], retriever=retriever
    )

    context_eval_results = []
    for cid, q in zip(expected_ids,queries):
        context_relev_eval = retriever_evaluator.evaluate(
            query=q, expected_ids=[cid,]
        )
        context_eval_results.append(context_relev_eval)

    eval_df = pd.DataFrame({
        'query': [cer.query for cer in context_eval_results],
        'expected_ids': [cer.expected_ids for cer in context_eval_results],
        'retrieved_ids': [cer.retrieved_ids for cer in context_eval_results],
        'mrr': [cer.metric_dict['mrr'].score for cer in context_eval_results],
        'hit_rate': [cer.metric_dict['hit_rate'].score for cer in context_eval_results]
    })

    print('Total MRR = ',eval_df.mrr.sum(),'/ ',len(queries))
    print('# Hits = ',eval_df.hit_rate.sum(),'/ ',len(queries))
    return eval_df

## First lets test on our sample Q&A pair
query_id = '0222418f-4e63-4577-84ec-94597552bff4'
corpus_id = val_dataset.__dict__['relevant_docs'][query_id][0]
sample_chunk = val_dataset.__dict__['corpus'][corpus_id]
sample_query = val_dataset.__dict__['queries'][query_id]

print('Query:\n', sample_query)
results = run_context_relevance_eval(index,[sample_query,],[corpus_id,])

print('Expected ID = ',results.iloc[0].expected_ids)
print('Retrieved IDs = ',results.iloc[0].retrieved_ids)
print('Top doc:\n',val_dataset.__dict__['corpus'][results.iloc[0].retrieved_ids[0]])

Query:
 How can individuals request corrections for factually inaccurate information about themselves in ChatGPT output?
Total MRR =  1.0 /  1
# Hits =  1.0 /  1
Expected ID =  ['f712129a-a58d-4e36-b62f-22ebfeda56a8']
Retrieved IDs =  ['f712129a-a58d-4e36-b62f-22ebfeda56a8', '1f95b363-1002-4f2d-bf17-ab4842714072']
Top doc:
 If you notice that ChatGPT output contains factually inaccurate information about you and you would like us to correct the inaccuracy, you may submit a correction request through privacy.openai.com or to dsar@openai.com. Given the technical complexity of how our models work, we may not be able to correct the inaccuracy in every instance. In that case, you may request that we remove your Personal Information from ChatGPT’s output by filling out this form.


In [ ]:
## Convert the val_dataset Q&A results to a dictionary linking each query
## to the document it was built from
query_ids = list(val_dataset.__dict__['queries'].keys())
corpus_ids = [val_dataset.__dict__['relevant_docs'][qid][0] for qid in query_ids]
queries = [val_dataset.__dict__['queries'][qid] for qid in query_ids]
expected_texts = [val_dataset.__dict__['corpus'][cid] for cid in corpus_ids]
## Create a label mapping from non-finetuned document indexing to fine-tuned doc
## indexing, so we know which document to expect when running the FT model.
index_ids = list(index.__dict__['_index_struct'].nodes_dict.keys())
index_ft_ids = list(index_ft.__dict__['_index_struct'].nodes_dict.keys())
ft_corpus_ids = {index_ids[i]: index_ft_ids[i] for i in range(len(index_ids))}

keys_df = pd.DataFrame({
    'query_ids': query_ids,
    'corpus_ids': corpus_ids,
    'ft_corpus_ids': [ft_corpus_ids[cid] for cid in corpus_ids],
    'queries': queries,
    'expected_texts': expected_texts
}).sample(50,random_state=7)

In [ ]:
results = run_context_relevance_eval(index,keys_df['queries'],keys_df['corpus_ids'],rerank=True)

Total MRR =  37.526923076923076 /  50
# Hits =  49.0 /  50


In [ ]:
results_ft = run_context_relevance_eval(index_ft,keys_df['queries'],keys_df['ft_corpus_ids'],rerank=True)

Total MRR =  40.7 /  50
# Hits =  49.0 /  50


## Answer Relevance -- Is the generated answer relevant to the query?

In [ ]:
## TruLens: Answer Relevance -- Is the answer relevant to the query?

from llama_index.core.evaluation import RelevancyEvaluator

def run_answer_relevance_eval(index,queries,rerank=False):

    if rerank:
        query_engine = index.as_query_engine(
            llm=llm_gpt,
            node_postprocessors=[
                LLMRerank(choice_batch_size=5, top_n=2,
            )],
        )
    else:
        query_engine = index.as_query_engine(llm=llm_gpt)
    ans_relev_evaluator = RelevancyEvaluator(llm=llm_gpt)

    answer_eval_results = []
    for query in tqdm(queries):
        response = query_engine.query(query)
        ans_relev_eval = ans_relev_evaluator.evaluate_response(query=query, response=response)
        answer_eval_results.append(ans_relev_eval)

    print(sum([aer.passing for aer in answer_eval_results]))
    return answer_eval_results

query = "How can individuals request corrections for factually inaccurate information about themselves in ChatGPT output?"
results = run_answer_relevance_eval(index,[query,],rerank=True)

print(results[0].response)
print(str(results[0].passing))

100%|██████████| 1/1 [00:04<00:00,  4.13s/it]

1
Individuals can request corrections for factually inaccurate information about themselves in ChatGPT output by submitting a correction request through privacy.openai.com or by sending an email to dsar@openai.com. If the inaccuracy cannot be corrected due to the technical complexity of the models, they can request the removal of their Personal Information from ChatGPT’s output by filling out a specific form.
True


In [ ]:
results = run_answer_relevance_eval(index,keys_df['queries'],rerank=True)

100%|██████████| 50/50 [03:24<00:00,  4.09s/it]

43


In [ ]:
results_ft = run_answer_relevance_eval(index_ft,keys_df['queries'],rerank=True)

100%|██████████| 50/50 [03:29<00:00,  4.19s/it]

47


# Groundedness -- Is the response supported by the context?

In [ ]:
from llama_index.core.evaluation import FaithfulnessEvaluator

def run_groundedness_eval(index,queries,rerank=False):

    if rerank:
        query_engine = index.as_query_engine(
            llm=llm_gpt,
            node_postprocessors=[
                LLMRerank(choice_batch_size=5, top_n=2,
            )],
        )
    else:
        query_engine = index.as_query_engine(llm=llm_gpt)
    grnd_eval = FaithfulnessEvaluator(llm=llm_gpt)

    # responses = [query_engine.query(q) for q in queries]
    # grnd_eval_results = [grnd_eval.evaluate_response(r) for r in responses]
    ground_eval_results = []
    for query in tqdm(queries):
        response = query_engine.query(query)
        ground_eval = faithfulness_eval.evaluate_response(response=response)
        ground_eval_results.append(ground_eval)

    print(sum([ger.passing for ger in ground_eval_results]))
    return ground_eval_results

query = "How can individuals request corrections for factually inaccurate information about themselves in ChatGPT output?"
results = run_answer_relevance_eval(index,[query,],rerank=True)

print('Contexts: ',results[0].contexts)
print('\nResponse: ',results[0].response)
print('\n',str(results[0].passing))

100%|██████████| 1/1 [00:03<00:00,  3.85s/it]

1
Contexts:  ['If you notice that ChatGPT output contains factually inaccurate information about you and you would like us to correct the inaccuracy, you may submit a correction request through privacy.openai.com or to dsar@openai.com. Given the technical complexity of how our models work, we may not be able to correct the inaccuracy in every instance. In that case, you may request that we remove your Personal Information from ChatGPT’s output by filling out this form.', 'If you are unable to exercise your rights through your account, please submit your request through privacy.openai.com or to dsar@openai.com.\n\nA note about accuracy: Services like ChatGPT generate responses by reading a user’s request and, in response, predicting the words most likely to appear next. In some cases, the words most likely to appear next may not be the most factually accurate. For this reason, you should not rely on the factual accuracy of output from our models.']

Response:  Individuals can request co

In [ ]:
results = run_groundedness_eval(index,keys_df['queries'],rerank=True)

100%|██████████| 50/50 [03:28<00:00,  4.17s/it]

47


In [ ]:
results_ft = run_groundedness_eval(index_ft,keys_df['queries'],rerank=True)

100%|██████████| 50/50 [03:29<00:00,  4.19s/it]

48


In [ ]:
## Create dataframes of the output to example results

grnd_df = pd.DataFrame({
    'query': keys_df['queries'],
    'contexts': [r.contexts for r in results],
    'response': [r.response for r in results],
    'pass': [str(r.passing) for r in results],
    'score': [r.score for r in results]
})

grnd_df_ft = pd.DataFrame({
    'query': keys_df['queries'],
    'contexts': [r.contexts for r in results_ft],
    'response': [r.response for r in results_ft],
    'pass': [str(r.passing) for r in results_ft],
    'score': [r.score for r in results_ft]
})